In [4]:
import pandas as pd
from datetime import datetime, timedelta
import copy
import random
import time

# --- 1. Definição do Problema e Parâmetros ---
SETUP_TIME_COR = 15
SETUP_TIME_PECA = 3
DAILY_CAPACITY_MINUTES = 18 * 60 + 35
VELOCIDADE_MONOVIA = 2.0  # metros/minuto

# --- 2. Funções do Algoritmo de Otimização (v2.1) ---

def calculate_cost_v2(sequence, initial_item=None):
    """Calcula o custo total de setup (cor e peça) de uma sequência."""
    if not sequence:
        return 0
    total_setup_cost = 0
    current_item = initial_item
    if current_item is not None:
        next_item = sequence[0]
        if current_item['Tinta'] != next_item['Tinta']:
            total_setup_cost += SETUP_TIME_COR
        if current_item['CODIGO_COMPONENTE'] != next_item['CODIGO_COMPONENTE']:
            total_setup_cost += SETUP_TIME_PECA
    for i in range(len(sequence) - 1):
        current_item = sequence[i]
        next_item = sequence[i+1]
        if current_item['Tinta'] != next_item['Tinta']:
            total_setup_cost += SETUP_TIME_COR
        if current_item['CODIGO_COMPONENTE'] != next_item['CODIGO_COMPONENTE']:
            total_setup_cost += SETUP_TIME_PECA
    return total_setup_cost

def calculate_prioritization_score(item):
    """Calcula a pontuação de prioridade para um item."""
    saldo = item['Estoque'] - item['Pedidos']
    due_date = item['Data_de_Entrega']
    prioridade_urgente = 1 if saldo >= 0 else 0
    prioridade_data = due_date.timestamp()
    prioridade_saldo = abs(saldo) if saldo < 0 else 0
    return (prioridade_urgente, prioridade_data, -prioridade_saldo)

def daily_sequencing_and_scheduling(unscheduled_items):
    """Cria a programação inicial usando a heurística de priorização."""
    schedule = []
    day_number = 1
    remaining_items = copy.deepcopy(unscheduled_items)
    while remaining_items:
        remaining_items.sort(key=calculate_prioritization_score)
        items_for_today_prioritized = []
        time_used_today_minutes = 0
        items_to_remove_from_main_list = []
        last_color_of_day = None
        for item in remaining_items:
            item_production_time_minutes = item['Tempo_Calculado'] * 60
            setup_cost_to_add = 0
            if items_for_today_prioritized:
                if last_color_of_day != item['Tinta']:
                    setup_cost_to_add = SETUP_TIME_COR
            if time_used_today_minutes + item_production_time_minutes + setup_cost_to_add <= DAILY_CAPACITY_MINUTES:
                items_for_today_prioritized.append(item)
                time_used_today_minutes += item_production_time_minutes
                if last_color_of_day != item['Tinta']:
                    time_used_today_minutes += setup_cost_to_add
                    last_color_of_day = item['Tinta']
                items_to_remove_from_main_list.append(item)
        if not items_for_today_prioritized:
            break
        color_groups = {}
        for item in items_for_today_prioritized:
            color = item['Tinta']
            if color not in color_groups:
                color_groups[color] = []
            color_groups[color].append(item)
        sorted_colors = sorted(color_groups.keys(), key=lambda c: calculate_prioritization_score(color_groups[c][0]))
        initial_daily_sequence = []
        for color in sorted_colors:
            initial_daily_sequence.extend(color_groups[color])
        current_day = {'day': day_number, 'items': initial_daily_sequence}
        schedule.append(current_day)
        remaining_items = [item for item in remaining_items if item not in items_to_remove_from_main_list]
        day_number += 1
    return schedule

def tabu_search_optimizer(daily_sequence, initial_item=None, tabu_tenure=7, max_iterations=100):
    """Otimiza a sequência de um único dia usando Busca Tabu."""
    current_solution = list(daily_sequence)
    best_solution = list(daily_sequence)
    best_cost = calculate_cost_v2(best_solution, initial_item)
    tabu_list = []
    for iteration in range(max_iterations):
        best_neighbor, best_neighbor_cost, best_move = None, float('inf'), None
        for i in range(len(current_solution)):
            for j in range(i + 1, len(current_solution)):
                neighbor = list(current_solution)
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                move = tuple(sorted((i, j)))
                neighbor_cost = calculate_cost_v2(neighbor, initial_item)
                if move not in tabu_list and neighbor_cost < best_neighbor_cost:
                    best_neighbor, best_neighbor_cost, best_move = neighbor, neighbor_cost, move
        if best_neighbor:
            current_solution = best_neighbor
            tabu_list.append(best_move)
            if len(tabu_list) > tabu_tenure:
                tabu_list.pop(0)
            if best_neighbor_cost < best_cost:
                best_solution, best_cost = best_neighbor, neighbor_cost
    return best_solution


# --- 3. Execução Principal ---

if __name__ == "__main__":
    try:
        # --- ETAPA DE PREPARAÇÃO DE DADOS ---
        print("Iniciando a preparação de dados...")

        path_estruturas = r'../data/processed/Planilha_Estruturas - Produto_Cor_Dim.csv'
        path_pedidos = r'../data/processed/pedidos.csv'
        
        df_estruturas = pd.read_csv(path_estruturas, sep=';', encoding='latin1')
        df_pedidos = pd.read_csv(path_pedidos, sep=',', encoding='latin1')

        cols_numericas = ['Pedidos', 'Estoque']
        for col in cols_numericas:
            df_pedidos[col] = pd.to_numeric(df_pedidos[col], errors='coerce').fillna(0).astype(int)
        df_pedidos['Data_Entrega'] = pd.to_datetime(df_pedidos['Data_Entrega'], errors='coerce', dayfirst=True)
        df_pedidos.dropna(subset=['Data_Entrega'], inplace=True)
        
        lista_tarefas_pintura = []
        for _, pedido in df_pedidos.iterrows():
            necessidade_producao = pedido['Pedidos'] - pedido['Estoque']
            if necessidade_producao > 0:
                componentes_do_produto = df_estruturas[df_estruturas['CODIGO_PRODUTO'] == pedido['CODIGO_PRODUTO']]
                for _, componente in componentes_do_produto.iterrows():
                    if pd.isna(componente['DESCRICAO_COR']):
                        continue
                    pecas_gancheira = pd.to_numeric(componente.get('Peças p/ gancheira', 1), errors='coerce')
                    espacamento = pd.to_numeric(componente.get('Espaçamento', 0.5), errors='coerce')
                    if pecas_gancheira and espacamento and pecas_gancheira > 0:
                        tempo_calculado_minutos = ((necessidade_producao / pecas_gancheira) * espacamento) / VELOCIDADE_MONOVIA
                    else:
                        tempo_calculado_minutos = 0
                    tarefa = {
                        'CODIGO_PRODUTO_FINAL': pedido['CODIGO_PRODUTO'],
                        'CODIGO_COMPONENTE': componente['CODIGO_COMPONENTE'],
                        'Tinta': componente['DESCRICAO_COR'],
                        'Quantidade_Planejada': necessidade_producao,
                        'Estoque': pedido['Estoque'],
                        'Pedidos': pedido['Pedidos'],
                        'Data_de_Entrega': pedido['Data_Entrega'],
                        'Tempo_Calculado': tempo_calculado_minutos / 60
                    }
                    lista_tarefas_pintura.append(tarefa)

        print(f"Preparação concluída. Total de {len(lista_tarefas_pintura)} lotes de componentes a serem planejados.")
        
        # --- ETAPA DE OTIMIZAÇÃO ---
        start_time = time.time()
        initial_schedule = daily_sequencing_and_scheduling(lista_tarefas_pintura)
        
        optimized_schedule = []
        print("Otimizando a sequência de cada dia com a Busca Tabu (v2.1)...")
        last_item_from_previous_day = None
        for day_data in initial_schedule:
            initial_daily_sequence = day_data['items']
            refined_daily_sequence = tabu_search_optimizer(initial_daily_sequence, initial_item=last_item_from_previous_day)
            refined_setup_cost = calculate_cost_v2(refined_daily_sequence, last_item_from_previous_day)
            production_time = sum(item['Tempo_Calculado'] * 60 for item in refined_daily_sequence)
            optimized_schedule.append({
                'day': day_data['day'],
                'items': refined_daily_sequence,
                'time_used_minutes': production_time + refined_setup_cost,
                'total_components': sum(item['Quantidade_Planejada'] for item in refined_daily_sequence),
                'setup_cost': refined_setup_cost
            })
            if refined_daily_sequence:
                last_item_from_previous_day = refined_daily_sequence[-1]
        print("Otimização concluída.")

        # --- ETAPA DE RESULTADOS (IMPRESSÃO NO CONSOLE) ---
        total_setup_optimized = sum(day['setup_cost'] for day in optimized_schedule)
        total_production_time_optimized = sum(day['time_used_minutes'] for day in optimized_schedule)
        end_time = time.time()

        print("\n" + "="*50 + "\n")
        print("Gerando o Cronograma de Produção Diário (Otimizado com Dados Reais):\n")
        for day in optimized_schedule:
            print(f"--- Dia {day['day']} ---")
            # ... (impressão dos detalhes do dia)
        
        print("\n" + "="*50 + "\n")
        print("Resumo do Agendamento Otimizado:")
        # ... (impressão do resumo)

        # --- NOVO: ETAPA DE EXPORTAÇÃO DOS ARQUIVOS ---
        print("\n" + "="*50 + "\n")
        print("Salvando o sequenciamento de cada dia em arquivos CSV e Excel...")

        for day in optimized_schedule:
            day_number = day['day']
            daily_items = day['items']
            
            # Cria um DataFrame pandas com os itens do dia
            df_daily_schedule = pd.DataFrame(daily_items)
            
            # Formata a coluna de data para melhor visualização nos arquivos
            if not df_daily_schedule.empty:
                 df_daily_schedule['Data_de_Entrega'] = df_daily_schedule['Data_de_Entrega'].dt.strftime('%d/%m/%Y')

            # Define os nomes dos arquivos
            csv_filename = f'cronograma_dia_{day_number}.csv'
            excel_filename = f'cronograma_dia_{day_number}.xlsx'
            
            # Salva em CSV
            df_daily_schedule.to_csv(csv_filename, sep=';', encoding='latin1', index=False)
            
            # Salva em Excel
            df_daily_schedule.to_excel(excel_filename, index=False)
            
            print(f" - Arquivos para o Dia {day_number} salvos: '{csv_filename}' e '{excel_filename}'")
            
        print("\nExportação concluída com sucesso!")
        print("="*50 + "\n")


    except FileNotFoundError as e:
        print(f"Erro: Arquivo não encontrado. Verifique o caminho: {e.filename}")
    except KeyError as e:
        print(f"Erro de Coluna: A coluna {e} não foi encontrada. Verifique se o nome da coluna no arquivo CSV está correto e se o separador (sep) está certo.")
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")

Iniciando a preparação de dados...
Preparação concluída. Total de 165 lotes de componentes a serem planejados.
Otimizando a sequência de cada dia com a Busca Tabu (v2.1)...
Otimização concluída.


Gerando o Cronograma de Produção Diário (Otimizado com Dados Reais):

--- Dia 1 ---
--- Dia 2 ---
--- Dia 3 ---
--- Dia 4 ---
--- Dia 5 ---


Resumo do Agendamento Otimizado:


Salvando o sequenciamento de cada dia em arquivos CSV e Excel...
 - Arquivos para o Dia 1 salvos: 'cronograma_dia_1.csv' e 'cronograma_dia_1.xlsx'
 - Arquivos para o Dia 2 salvos: 'cronograma_dia_2.csv' e 'cronograma_dia_2.xlsx'
 - Arquivos para o Dia 3 salvos: 'cronograma_dia_3.csv' e 'cronograma_dia_3.xlsx'
 - Arquivos para o Dia 4 salvos: 'cronograma_dia_4.csv' e 'cronograma_dia_4.xlsx'
 - Arquivos para o Dia 5 salvos: 'cronograma_dia_5.csv' e 'cronograma_dia_5.xlsx'

Exportação concluída com sucesso!

